# Importing Libraries

In [ ]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm.notebook as tqdm


# Loading Root Files Into Pandas Dataframes

In [ ]:
# replace this with a path to the WC ntuple files
wc_file_location = "/Users/leehagaman/data/processed_checkout_rootfiles/"

# let's open a nu_overlay file and examine the contents

f_nu_overlay = uproot.open(wc_file_location + "checkout_prodgenie_bnb_nu_overlay_run1_PF.root")
f_nu_overlay.keys()


### We see that there are several TTrees inside the wcpselection TDirectory. Now let's print the variables in each TTree.

In [ ]:
already_printed_keys = []

for key in f_nu_overlay["wcpselection"].items():
    key_name = key[0].split(";")[0]
    if key_name not in already_printed_keys: # sometimes a TTree is repeated, we ignore duplicates
        print("\nPrinting variables in TTree: ", key[0].split(";")[0])
        print(f_nu_overlay["wcpselection"][key[0]].keys())
        already_printed_keys.append(key_name)

### That's a lot of variables! Let's only load a few of the most relevant ones for now.

## Choosing Variables To Load

In [ ]:
T_bdt_vars = [              # variables involved with BDT training
    "nue_score",                    # BDT score for nue selection, used for the WC inclusive nueCC analysis
    "numu_score",                   # BDT score for numu selection, used for the WC inclusive numuCC selections
]

T_eval_vars = [             # variables involved with low level reconstruction and truth information
    "run",                          # run number
    "subrun",                       # subrun number
    "event",                        # event number
    "match_isFC",                   # reconstructed cluster is fully contained (FC), boolean
    "truth_nuEnergy",               # true neutrino energy (MeV)
    "truth_nuPdg",                  # true neutrino pdg code
    "truth_isCC",                   # true interaction type is charged current, boolean
    "match_completeness_energy",    # the true energy deposited in the clusters that are 3D-matched with the reconstructed neutrino clusters (MeV)
    "truth_energyInside",           # the true energy deposited in the TPC Fiducial Volume (MeV)
    "truth_vtxInside",              # boolean, true neutrino vertex is inside the TPC Fiducial Volume
    "truth_vtxX",                   # true neutrino vertex x (cm)
    "truth_vtxY",                   # true neutrino vertex y (cm)
    "truth_vtxZ",                   # true neutrino vertex z (cm)
    "weight_cv",                    # untuned GENIE event weight
    "weight_spline",                # additional MicroBooNE Tune weight
]
T_eval_data_vars = [        # same as above, but for data files we do not attempt to load any truth information
    "match_isFC",
]

T_kine_vars = [             # variables involved with kinematic reconstruction
    "kine_reco_Enu",                # reconstructed neutrino energy (MeV)   
]

T_pf_vars = [               # variables involved with individual particles
    "truth_NprimPio",
    "truth_NCDelta",
    "reco_nuvtxX",
    "reco_nuvtxY",
    "reco_nuvtxZ",
    "reco_muonMomentum",            # reconstructed muon momentum 4-vector (p_x, p_y, p_z, p_t), in (GeV/c, GeV/c, GeV/c, GeV)

    # These variables are related to individual true particles
    "truth_Ntrack",
    "truth_id",
    "truth_pdg",
    "truth_mother",
    "truth_startMomentum",
    "truth_startXYZT",

    # These variables are related to individual reco particles
    "reco_Ntrack",
    "reco_id",
    "reco_pdg",
    "reco_mother",
    "reco_startMomentum",
    "reco_startXYZT",
]
T_pf_data_vars = [          # same as above, but for data files we do not attempt to load any truth information
    "reco_nuvtxX",
    "reco_nuvtxY",
    "reco_nuvtxZ",
    "reco_muonMomentum",
    "reco_Ntrack",
    "reco_id",
    "reco_pdg",
    "reco_mother",
    "reco_startMomentum",
    "reco_startXYZT",
]


### Loading nu_overlay File

In [ ]:
f_nu_overlay = uproot.open(wc_file_location + "checkout_prodgenie_bnb_nu_overlay_run1_PF.root") # loading the nu_overlay file

# loading variables from each TTree
nu_overlay_bdt_df = f_nu_overlay["wcpselection"]["T_BDTvars"].arrays(T_bdt_vars, library="pd")
nu_overlay_eval_df = f_nu_overlay["wcpselection"]["T_eval"].arrays(T_eval_vars, library="pd")
nu_overlay_kine_df = f_nu_overlay["wcpselection"]["T_KINEvars"].arrays(T_kine_vars, library="pd")
nu_overlay_pf_df = f_nu_overlay["wcpselection"]["T_PFeval"].arrays(T_pf_vars, library="pd")

# combining everything into a single dataframe
nu_overlay_df = pd.concat([nu_overlay_bdt_df, nu_overlay_eval_df, nu_overlay_kine_df, nu_overlay_pf_df], axis=1)

# deleting temporary dataframes to free up memory
del nu_overlay_bdt_df, nu_overlay_eval_df, nu_overlay_kine_df, nu_overlay_pf_df

nu_overlay_df

### Loading Data, Dirt, and EXT files

# Making A Histogram

# Plotting An Efficiency

# Particle-level Information